### <center>Очистка данных (практика)

Вам предоставлен набор данных, первоначально полученный в Национальном институте диабета, болезней органов пищеварения и почек. 

Этот набор данных создан для того, чтобы на основе определённых диагностических измерений предсказать, есть ли у пациента диабет. 

На выбор этих экземпляров из более крупной базы данных было наложено несколько ограничений. В частности, все пациенты здесь — женщины не моложе 21 года индейского происхождения Пима.

Прочитаем наши данные и выведем первые пять строк таблицы:

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
diabetes = pd.read_csv('data/diabetes_data.csv')
diabetes.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,Gender
0,6,98,58,33,190,34.0,0.430,43,0,Female
1,2,112,75,32,0,35.7,0.148,21,0,Female
2,2,108,64,0,0,30.8,0.158,21,0,Female
3,8,107,80,0,0,24.6,0.856,34,0,Female
4,7,136,90,0,0,29.9,0.210,50,0,Female


##### Признаки данных

Pregnancies — количество беременностей.

Glucose — концентрация глюкозы в плазме через два часа при пероральном тесте на толерантность к глюкозе.

BloodPressure — диастолическое артериальное давление (мм рт. ст.).

SkinThickness — толщина кожной складки трицепса (мм).

Insulin — двухчасовой сывороточный инсулин (ме Ед/мл).

BMI — индекс массы тела (вес в кг/(рост в м)²).

DiabetesPedigreeFunction — функция родословной диабета (чем она выше, тем выше шанс наследственной заболеваемости).

Age — возраст.

Outcome — наличие диабета (0 — нет, 1 — да).

##### Задание 8.1

Начнём с поиска дубликатов в данных. Найдите все повторяющиеся строки в данных и удалите их. Для поиска используйте все признаки в данных. Сколько записей осталось в данных?

In [4]:
dupl_columns = list(diabetes.columns)
mask = diabetes.duplicated(subset=dupl_columns)
diabetes_duplicates = diabetes[mask]
print(mask)
print(f'Число найденных дубликатов: {diabetes_duplicates.shape[0]}')

0      False
1      False
2      False
3      False
4      False
       ...  
773     True
774     True
775     True
776     True
777     True
Length: 778, dtype: bool
Число найденных дубликатов: 10


In [5]:
diabetes_dedupped = diabetes.drop_duplicates(subset=dupl_columns)
print(f'Результирующее число записей: {diabetes_dedupped.shape[0]}')

Результирующее число записей: 768


##### Задание 8.2

Далее найдите все неинформативные признаки в данных и избавьтесь от них. В качестве порога информативности возьмите 0.95: удалите все признаки, для которых 95 % значений повторяются или 95 % записей уникальны. В ответ запишите имена признаков, которые вы нашли (без кавычек).

In [6]:
#список неинформативных признаков
low_information_cols = [] 

#цикл по всем столбцам
for col in diabetes_dedupped.columns:
    #наибольшая относительная частота в признаке
    top_freq = diabetes_dedupped[col].value_counts(normalize=True).max()
    #доля уникальных значений от размера признака
    nunique_ratio = diabetes_dedupped[col].nunique() / diabetes_dedupped[col].count()
    # сравниваем наибольшую частоту с порогом
    if top_freq > 0.95:
        low_information_cols.append(col)
        print(f'{col}: {round(top_freq*100, 2)}% одинаковых значений')
    # сравниваем долю уникальных значений с порогом
    if nunique_ratio > 0.95:
        low_information_cols.append(col)
        print(f'{col}: {round(nunique_ratio*100, 2)}% уникальных значений')

Gender: 100.0% одинаковых значений


In [7]:
information_diabetes_dedupped = diabetes_dedupped.drop(low_information_cols, axis=1)
print(f'Результирующее число признаков: {information_diabetes_dedupped.shape[1]}')

Результирующее число признаков: 9


##### Задание 8.3

Попробуйте найти пропуски в данных с помощью метода isnull().

Спойлер: ничего не найдёте. А они есть! Просто они скрыты от наших глаз. В таблице пропуски в столбцах Glucose, BloodPressure, SkinThickness, Insulin и BMI обозначены нулём, поэтому традиционные методы поиска пропусков ничего вам не покажут. Давайте это исправим!

Замените все записи, равные 0, в столбцах Glucose, BloodPressure, SkinThickness, Insulin и BMI на символ пропуска. Его вы можете взять из библиотеки numpy: np.nan.

Какая доля пропусков содержится в столбце Insulin? Ответ округлите до сотых.

In [8]:
columns_to_replace = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']

# Функция для замены нулевых значений на np.nan
def replace_zero_with_nan(value):
    return np.nan if value == 0 else value

# Применение функции к нужным столбцам
for column in columns_to_replace:
    information_diabetes_dedupped[column] = information_diabetes_dedupped[column].apply(replace_zero_with_nan)

# Вывод обновленных данных
print(information_diabetes_dedupped)

     Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI   
0              6     98.0           58.0           33.0    190.0  34.0  \
1              2    112.0           75.0           32.0      NaN  35.7   
2              2    108.0           64.0            NaN      NaN  30.8   
3              8    107.0           80.0            NaN      NaN  24.6   
4              7    136.0           90.0            NaN      NaN  29.9   
..           ...      ...            ...            ...      ...   ...   
763            5    139.0           64.0           35.0    140.0  28.6   
764            1     96.0          122.0            NaN      NaN  22.4   
765           10    101.0           86.0           37.0      NaN  45.6   
766            0    141.0            NaN            NaN      NaN  42.4   
767            0    125.0           96.0            NaN      NaN  22.5   

     DiabetesPedigreeFunction  Age  Outcome  
0                       0.430   43        0  
1                  

In [12]:
cols_null_percent = information_diabetes_dedupped.isnull().mean() 
cols_with_null = cols_null_percent[cols_null_percent>0].sort_values(ascending=False)
display(round(cols_with_null,2))

Insulin          0.49
SkinThickness    0.30
BloodPressure    0.05
BMI              0.01
Glucose          0.01
dtype: float64

##### Задание 8.4

Удалите из данных признаки, где число пропусков составляет более 30 %. Сколько признаков осталось в ваших данных (с учетом удаленных неинформативных признаков в задании 8.2)?

In [14]:
#создаём копию исходной таблицы
dedupped_2 = information_diabetes_dedupped.copy()

#отбрасываем столбцы с числом пропусков более 30% (100-70)
n = dedupped_2.shape[0] #число строк в таблице
thresh = n*0.7
dedupped_2 = dedupped_2.dropna(thresh=thresh, axis=1)
#выводим результирующую долю пропусков
display(dedupped_2)
print(f'Результирующее число признаков: {dedupped_2.shape[1]}')

,Pregnancies,Glucose,BloodPressure,SkinThickness,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,98.0,58.0,33.0,34.0,0.430,43,0
1,2,112.0,75.0,32.0,35.7,0.148,21,0
2,2,108.0,64.0,NaN,30.8,0.158,21,0
3,8,107.0,80.0,NaN,24.6,0.856,34,0
4,7,136.0,90.0,NaN,29.9,0.210,50,0
...,...,...,...,...,...,...,...,...
763,5,139.0,64.0,35.0,28.6,0.411,26,0
764,1,96.0,122.0,NaN,22.4,0.207,27,0
765,10,101.0,86.0,37.0,45.6,1.136,38,1
766,0,141.0,NaN,NaN,42.4,0.205,29,1


Результирующее число признаков: 8


##### Задание 8.5

Удалите из данных только те строки, в которых содержится более двух пропусков одновременно. Чему равно результирующее число записей в таблице?

In [16]:
#отбрасываем строки с числом пропусков более 2 в строке
m = dedupped_2.shape[1] #число признаков после удаления столбцов
dedupped_2 = dedupped_2.dropna(thresh=m-2, axis=0)
#выводим результирующую долю пропусков
display(dedupped_2)
print(f'Результирующее число строк: {dedupped_2.shape[0]}')

,Pregnancies,Glucose,BloodPressure,SkinThickness,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,98.0,58.0,33.0,34.0,0.430,43,0
1,2,112.0,75.0,32.0,35.7,0.148,21,0
2,2,108.0,64.0,NaN,30.8,0.158,21,0
3,8,107.0,80.0,NaN,24.6,0.856,34,0
4,7,136.0,90.0,NaN,29.9,0.210,50,0
...,...,...,...,...,...,...,...,...
763,5,139.0,64.0,35.0,28.6,0.411,26,0
764,1,96.0,122.0,NaN,22.4,0.207,27,0
765,10,101.0,86.0,37.0,45.6,1.136,38,1
766,0,141.0,NaN,NaN,42.4,0.205,29,1


Результирующее число строк: 761


##### Задание 8.6